# Mapping using Python

Here we use geotext library to extract places names from the sentence.

## Installation of geotext modules

In [10]:
!pip install geotext

### Example to use GeoText module

In [11]:
from geotext import GeoText

places = GeoText("I was in Chennai on that day and next day i went to Erode")
places.cities

['Chennai', 'Erode']

## Example to Exract place names from a website

* here we going to extract place names from the caption of the picture on the site , "https://www.gutenberg.org/files/64316/64316-h/64316-h.htm"

* also we used the method of previous web scraping method using BeautifulSoup module


![](../images/map2.png)

In [12]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
from geotext import GeoText

In [13]:
url="https://www.gutenberg.org/files/64316/64316-h/64316-h.htm"

In [266]:
page=requests.get(url)
     
soup=BeautifulSoup(page.text,'html.parser')
frame=[]
resultframe=[]
links=soup.find_all('div',attrs={'class':'ills'})
#links=soup.find_all('li',attrs={'class':'o-listicle__item'})
#print(links)
filename="caption.csv"
f=open(filename,"w", encoding = 'utf-8')
headers="Link,Caption\n"
f.write(headers)
u='https://www.gutenberg.org/files/64316/64316-h/'
for i in links:
    k=i.find_all('div',attrs={'class':'figcenter'})
    for j in k:
        l=j.find_all('a')
        Link = u+l[1].find('img')['src'].strip()          #Extract Link of all the images
        cap=j.find('p',attrs={'class':'caption'}).text.strip()  #Extract caption of all the images
        
        
        a=cap.replace(' ',',').title()

        result = GeoText(a,'US').cities
        
        if(result!=[]):
            frame.append((Link,str(result[0])))  
            f.write(Link+","+str(result[0])+"\n")   #stored only the link and caption that contains city names
    resultframe.extend(frame)
f.close()
        

### Result of the above Code

In [14]:
data=pd.DataFrame(resultframe, columns=['Link','Caption'])
print(data)

                                                Link      Caption
0  https://www.gutenberg.org/files/64316/64316-h/...  Westminster
1  https://www.gutenberg.org/files/64316/64316-h/...   Birmingham
2  https://www.gutenberg.org/files/64316/64316-h/...         Kent
3  https://www.gutenberg.org/files/64316/64316-h/...    Rotterdam
4  https://www.gutenberg.org/files/64316/64316-h/...  Westminster


### Installing arcgis to map the places

In [214]:
!pip install arcgis

Processing c:\users\mohanapriya\appdata\local\pip\cache\wheels\4d\11\58\7d0a04db6c902ef42b717da2981807529f4922485141ab404f\lerc-0.1.0-py3-none-any.whl
  Using cached ujson-4.0.1-cp38-cp38-win_amd64.whl (44 kB)
  Using cached python_certifi_win32-1.6-py2.py3-none-any.whl (7.2 kB)
Processing c:\users\mohanapriya\appdata\local\pip\cache\wheels\1f\1b\b5\54affbefc8a7e2bdf1da000fc576b8a1c91338f1f327a04f4c\pyshp-2.1.3-py3-none-any.whl
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached requests_toolbelt-0.9.1-py2.py3-none-any.whl (54 kB)
  Using cached requests_ntlm-1.1.0-py2.py3-none-any.whl (5.7 kB)
  Using cached requests_negotiate_sspi-0.5.2-py2.py3-none-any.whl (7.1 kB)
  Using cached requests_kerberos-0.12.0-py2.py3-none-any.whl (14 kB)
  Using cached winkerberos-0.7.0-cp38-cp38-win_amd64.whl (23 kB)
  Using cached setuptools_scm-5.0.1-py2.py3-none-any.whl (28 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)
  Using cached ntlm_auth-1.5.0-py

## Code to find latitude and longitude
here we find the latitude and longitude of the places that extract in the caption of the images. and we mark the point on the map using simple loop condition. 

In [16]:
from arcgis.gis import GIS
from arcgis.geometry import Point
from arcgis.geocoding import geocode, reverse_geocode

my_gis = GIS()
# m = my_gis.map()

m = my_gis.map("US", zoomlevel=4)


# function to find the latitude and longitude
def place_names(get_places):
    i=get_places
    geocoded=geocode(i)[0]['location']
    return geocoded

for i in data['Caption']:

    geocoded=place_names(i)
    lat=geocoded['y']
    longi=geocoded['x']
    print(lat,longi)
    
    location = {
     'Y': lat,                 # `Y` is latitude
     'X':  longi,               # `X` is longitude
     'spatialReference': {
         'wkid':4326
     }
    }
    unknown_pt = Point(location)
    address = reverse_geocode(location=unknown_pt)
    print(address)  #matching address found from the latitude and longitude
   
    m.draw(address)
  
   

39.86347000000006 -105.05000999999999
{'address': {'Match_addr': '4890-4998 W 92nd Ave, Westminster, Colorado, 80031', 'LongLabel': '4890-4998 W 92nd Ave, Westminster, CO, 80031, USA', 'ShortLabel': '4890-4998 W 92nd Ave', 'Addr_type': 'StreetAddress', 'Type': '', 'PlaceName': '', 'AddNum': '4944', 'Address': '4944 W 92nd Ave', 'Block': '', 'Sector': '', 'Neighborhood': 'South Central Westminster', 'District': '', 'City': 'Westminster', 'MetroArea': 'Denver Metro Area', 'Subregion': 'Adams County', 'Region': 'Colorado', 'Territory': '', 'Postal': '80031', 'PostalExt': '', 'CountryCode': 'USA'}, 'location': {'x': -105.05000999999999, 'y': 39.863412756606785, 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}}
52.47891000000004 -1.9059199999999805
{'address': {'Match_addr': 'Paradise Circus, Birmingham, West Midlands, England, B3 3', 'LongLabel': 'Paradise Circus, Birmingham, West Midlands, England, B3 3, GBR', 'ShortLabel': 'Paradise Circus', 'Addr_type': 'StreetName', 'Type': '', 

### Result Map of the above code

In [17]:
m

MapView(layout=Layout(height='400px', width='100%'))

![](../images/resmap1.png)

## Example to Find the address of the Given latitude and longitude

In [19]:
from arcgis.geometry import Point
location = {
     'Y': 39.86347000000006,                 # `Y` is latitude
     'X':  -105.05000999999999,               # `X` is longitude
     'spatialReference': {
         'wkid':4326
     }
}
unknown_pt = Point(location)
address = reverse_geocode(location=unknown_pt)
address

{'address': {'Match_addr': '4890-4998 W 92nd Ave, Westminster, Colorado, 80031',
  'LongLabel': '4890-4998 W 92nd Ave, Westminster, CO, 80031, USA',
  'ShortLabel': '4890-4998 W 92nd Ave',
  'Addr_type': 'StreetAddress',
  'Type': '',
  'PlaceName': '',
  'AddNum': '4944',
  'Address': '4944 W 92nd Ave',
  'Block': '',
  'Sector': '',
  'Neighborhood': 'South Central Westminster',
  'District': '',
  'City': 'Westminster',
  'MetroArea': 'Denver Metro Area',
  'Subregion': 'Adams County',
  'Region': 'Colorado',
  'Territory': '',
  'Postal': '80031',
  'PostalExt': '',
  'CountryCode': 'USA'},
 'location': {'x': -105.05000999999999,
  'y': 39.863412756606785,
  'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}}

### Draw a map for above address

In [21]:
m = my_gis.map("USA", zoomlevel=5)
m.draw(address)
m

MapView(layout=Layout(height='400px', width='100%'))

![](../images/resmap2.png)

## map can be save as html file

In [22]:
file_path = r"C:\Users\Mohanapriya\city_map.html"
m.export_to_html(file_path)

True

In [333]:
import webbrowser
webbrowser.open_new('http://127.0.0.1:2000/')

True